In [5]:
#fast text. using: very simple model;n-gram to captrue location information;h-softmax to speed up training/inference
print("started...")
class fastTextBright:
    def __init__(self,label_size,learning_rate,batch_size,decay_steps, decay_rate):
        """init all hyperparameter here"""
        #set hyperparamter
        self.label_size=label_size
        self.batch_size=batch_size
        self.num_sampled=num_sampled
        
        # add placeholder (X,label)
        self.sentence=tf.placeholder(tf.int32,[None,self.sentence_len],name="sentence") #X
        self.labels=tf.placeholder(tf.int64,[None],name="Labels") #y
        
        self.global_step = tf.Variable(0, trainable=False, name="Global_Step")
        self.decay_steps, self.decay_rate =decay_steps, decay_rate
        
        instantiate_weights()
        self.logits=inference()
        self.loss_val=self.loss()
        self.train_op=self.train()
        correct_prediction=tf.equal(tf.argmax(self.logits,1),self.A)
        self.accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32),name="Accuracy")
        
    def instantiate_weights(self):
        """define all weights here"""
        #embedding matrix
        self.Embedding=tf.get_variable("Embedding",[self.vocab_size,self.embed_size])
        self.W=tf.get_variable("W",[self.embed_size,self.label_size])
        self.b=tg.get_variable("b",[self.label_size])
    
    def inference(self):
        """main computation graph here: embedding-->average-->linear classifier"""
        #1.get emebedding of words in the sentence
        sentence_embeddings=tf.nn.embedding_lookup(self.Embedding,self.sentence)#[None,self.sentence_len,self.embed_size]
        #2.average vectors, to get representation of the sentence
        sentence_embeddings=tf.reduce_mean(sentence_embeddings,axis=1) #[None,self.embed_size]
        #3.linear classifier layer
        #[None,self.embed_size],[self.embed_size,self.label_size]===>[None, self.label_size]
        logits=tf.matmul(sentence_embeddings,self.W)+self.b
        return logits
    
    def loss(self):
        """calculate loss using (NCE)cross entropy here"""
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(
                   tf.nn.nce_loss(weights=self.W, #nce_weights,
                   biases=self.b, #nce_biases,
                   labels=self.labels, #train_labels,
                   inputs=self.Embedding,#embed,
                   num_sampled=num_sampled,
                   num_classes=label_size)) #vocabulary_size))
        return loss
    
    def train(self):
        """based on the loss, use SGD to update parameter"""
        learning_rate=tf.train.exponential_decay(self.learning_rate,self.global_step,self.decay_steps,
                                                self.decay_rate,staircase=True)
        train_op=tf.contrib.layers.optimize_loss(self.loss_val,global_step=self.global_step,
                                                 learning_rate=learning_rate,optimizer="Adam")
        return train_op
    
print("ended...")   

started...
ended...


In [8]:
# Using tflearn for a simple classification problem
print("started...")
import tflearn
tflearn.init_graph(num_cores=8, gpu_memory_fraction=0.5)

net = tflearn.input_data(shape=[None, 784])
net = tflearn.fully_connected(net, 64)
net = tflearn.dropout(net, 0.5)
net = tflearn.fully_connected(net, 10, activation='softmax')
net = tflearn.regression(net, optimizer='adam', loss='categorical_crossentropy')

model = tflearn.DNN(net)
model.fit(X, Y)
print("ended...")

started...


ImportError: No module named '_curses'